In [ ]:
import pandas as pd
from sklearn.ensemble import IsolationForest
from sklearn.cluster import DBSCAN
import pyarrow as pa
import pyarrow.parquet as pq
import gcsfs
import getpass
import os
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import StandardScaler, OneHotEncoder, RobustScaler
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.metrics import mean_squared_error, mean_absolute_error, median_absolute_error, r2_score
import geopandas as gpd
from scikeras.wrappers import KerasRegressor
import tensorflow as tf
import matplotlib.pyplot as plt
# import sgis as sg
import dapla as dp
import datetime
from dapla.auth import AuthClient
from dapla import FileClient
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
import requests
from pyjstat import pyjstat
# import plotly.express as px
from ipywidgets import interact, Dropdown
# from klass import search_classification

import sys

sys.path.append("../functions")
import kommune_pop
import kommune_inntekt
import kpi
import ao
import kommune_translate
import kommune

fs = FileClient.get_gcs_file_system()
import numpy as np


import warnings

warnings.filterwarnings("ignore")

import math

# good_df = ao.rette_bedrifter(good_df)

import input_data
# import create_datafiles

from joblib import Parallel, delayed
import multiprocessing

import time
import create_datafiles

In [ ]:
# search_classification()

year = 2021

In [ ]:
start_time = time.time()
current_year_good_oms, current_year_bad_oms, v_orgnr_list_for_imputering, training_data, imputatable_df, time_series_df = create_datafiles.main(year, 0.65)

processing_time = time.time() - start_time
print(f"Time taken to create training data: {processing_time:.2f} seconds")

##### Fix Early Stopping/ implement flexible learning rate


In [ ]:
import numpy as np
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.neighbors import KNeighborsRegressor
import matplotlib.pyplot as plt

def knn_model(training_df, scaler, df_estimeres, GridSearch=True):
    
    df = training_df.copy()
    imputed_df = df_estimeres.copy()

    columns_to_fill = ["nacef_5", "tmp_sn2007_5", "b_kommunenr"]
    numeric_columns_to_fill = [
        "inntekt_delta_oms",
        "emp_delta_oms",
        "befolkning_delta_oms",
        "inflation_rate_oms",
        "gjeldende_bdr_syss",
        "new_oms_trendForecast",
        'oms_syssmean_basedOn_naring',
        'oms_syssmean_basedOn_naring_kommune'
    ]

    # Fill NaN values with 'missing' for the specified columns
    df[columns_to_fill] = df[columns_to_fill].fillna('missing')
    imputed_df[columns_to_fill] = imputed_df[columns_to_fill].fillna('missing')
    
    df[numeric_columns_to_fill] = df[numeric_columns_to_fill].fillna(0)
    imputed_df[numeric_columns_to_fill] = imputed_df[numeric_columns_to_fill].fillna(0)

    categorical_columns = ["nacef_5", "tmp_sn2007_5", "b_kommunenr"]
    for col in categorical_columns:
        df[col] = df[col].astype("category")

    X = df.drop(columns=["new_oms"])
    y = df["new_oms"]

    categorical_features = ["nacef_5", "tmp_sn2007_5", "b_kommunenr"]
    numerical_features = [
        "inntekt_delta_oms",
        "emp_delta_oms",
        "befolkning_delta_oms",
        "inflation_rate_oms",
        "gjeldende_bdr_syss",
        "new_oms_trendForecast",
        'oms_syssmean_basedOn_naring',
        'oms_syssmean_basedOn_naring_kommune'
    ]

    preprocessor = ColumnTransformer(
        transformers=[
            ("num", scaler, numerical_features),
            ("cat", OneHotEncoder(categories="auto", handle_unknown="ignore"), categorical_features),
        ]
    )

    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    preprocessor.fit(X_train)
    X_train_transformed = preprocessor.transform(X_train)
    X_test_transformed = preprocessor.transform(X_test)

    if GridSearch:
        # Define the model
        regressor = KNeighborsRegressor()

        # Define parameter grid for GridSearch
        param_grid = {
            'n_neighbors': [2, 3, 5, 7]
        }

        # Grid search with cross-validation
        grid_search = GridSearchCV(estimator=regressor, param_grid=param_grid, scoring='neg_mean_squared_error', cv=3, verbose=1)
        grid_search.fit(X_train_transformed, y_train)

        # Print best parameters
        print("Best parameters found by GridSearch:", grid_search.best_params_)

        # Use best estimator from grid search
        regressor = grid_search.best_estimator_
    else:
        # Define the model with default parameters
        regressor = KNeighborsRegressor(n_neighbors=5)

        # Train the model
        regressor.fit(X_train_transformed, y_train)

    y_pred = regressor.predict(X_test_transformed)

    mse = mean_squared_error(y_test, y_pred)
    r_squared = r2_score(y_test, y_pred)
    mae = mean_absolute_error(y_test, y_pred)
    print("Mean Squared Error:", mse)
    print("R-squared:", r_squared)
    print("Mean Absolute Error:", mae)
    
    # Plot Predicted vs. Actual Values
    plt.figure(figsize=(10, 5))
    plt.scatter(y_test, y_pred, alpha=0.3)
    plt.plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()], "r--", lw=2)
    plt.xlabel("Actual")
    plt.ylabel("Predicted")
    plt.title("Predicted vs. Actual Values")
    plt.show()

    # Plot Residuals
    residuals = y_test - y_pred
    plt.figure(figsize=(10, 5))
    plt.scatter(y_test, residuals, alpha=0.3)
    plt.hlines(0, y_test.min(), y_test.max(), colors="r", linestyles="dashed")
    plt.xlabel("Actual")
    plt.ylabel("Residuals")
    plt.title("Residuals Plot")
    plt.show()

    imputed_X = imputed_df.drop(columns=["new_oms"])
    imputed_X_transformed = preprocessor.transform(imputed_X)
    imputed_df["predicted_oms"] = regressor.predict(imputed_X_transformed)
    
    return imputed_df

In [ ]:
imputed_df = knn_model(training_data, RobustScaler(), imputatable_df, GridSearch=False)

In [ ]:
# sort predicted_oms from smallest to larges
test = imputed_df.copy()
test.sort_values(by="predicted_oms", ascending=True, inplace=True)

test.head()


In [ ]:
# df_to_merge = imputed_df[['v_orgnr', 'year', 'b_kommunenr', 'orgnr_foretak', 'naring', 'predicted_oms']]
df_to_merge = imputed_df[['v_orgnr', 'year', 'id', 'predicted_oms']]
# df_to_merge.rename(columns={'orgnr_foretak': 'orgnr_n_1', 'naring':'tmp_sn2007_5'}, inplace=True)

In [ ]:
test = df_to_merge.copy()
test.sort_values(by="predicted_oms", ascending=True, inplace=True)

test.head()

In [ ]:
bad_df = pd.merge(current_year_bad_oms, df_to_merge, on=['v_orgnr', 'id', 'year'], how='left')

test = bad_df.copy()
test.sort_values(by="predicted_oms", ascending=True, inplace=True)

# change pd option to show all columns
pd.set_option('display.max_columns', None)

test.head()

In [ ]:
df = bad_df[(bad_df['predicted_oms'].isnull() | np.isinf(bad_df['predicted_oms']))]
df.head(50)

In [ ]:


bad_df['new_oms'] = bad_df['predicted_oms']

bad_df.drop(['predicted_oms'], axis=1, inplace=True)

good_df = pd.concat([current_year_good_oms, bad_df], ignore_index=True)

# filter out good_df so that lopenr == 1
good_df = good_df[good_df['lopenr'] == 1]

print(good_df.shape)

# if 'new_oms' is less than 0 then 'new_oms' = 0
good_df['new_oms'] = good_df['new_oms'].apply(lambda x: 0 if x < 0 else x)


good_df.drop(['tot_oms_fordelt'], axis=1, inplace=True)

good_df.head()

In [ ]:
test = good_df.copy()
test.sort_values(by="new_oms", ascending=True, inplace=True)

test.head()

In [ ]:
# Group by 'id' and calculate the sum
grouped = (
    good_df.groupby("id")[["new_oms"]].sum().reset_index()
)

In [ ]:
# Rename the columns
grouped.rename(
    columns={"new_oms": "tot_oms_fordelt"},
    inplace=True,
)

In [ ]:
# Merge the grouped DataFrame back to the original DataFrame based on 'id'
good_df = pd.merge(good_df, grouped, on="id", how="left")

good_df['oms_share'] = good_df['new_oms'] / good_df['tot_oms_fordelt']

# drop 'new_oms' column
good_df.drop(['new_oms', 'tot_oms_fordelt'], axis=1, inplace=True)

good_df["oms_share"].replace([np.inf, -np.inf], np.nan, inplace=True)
good_df['oms_share'].fillna(0, inplace=True)

good_df['new_oms'] = good_df['oms_share'] * good_df['foretak_omsetning']

In [ ]:
good_df.head()

In [ ]:
#### TEST
test_grouped = (
    good_df.groupby("id")[["new_oms"]].sum().reset_index()
)

test_grouped.rename(
    columns={"new_oms": "test_tot_oms_fordelt"},
    inplace=True,
)

test = pd.merge(good_df, test_grouped, on="id", how="left")

# Convert to integers
good_df['foretak_omsetning'] = good_df['foretak_omsetning'].astype(float).round().astype(int)
good_df['new_oms'] = good_df['new_oms'].astype(float).round().astype(int)

test['oms_diff'] = test['foretak_omsetning'] - test['test_tot_oms_fordelt']

# filter test for results where oms_diff is not 0
test = test[test['oms_diff'] != 0]

test['oms_diff'] = test['oms_diff'].abs()

# Step 2: Sort the DataFrame by oms_diff in descending order
test_sorted = test.sort_values(by='oms_diff', ascending=False)

test.head()

In [ ]:
good_df['id'] = good_df['id'].astype(str)
good_df['nacef_5'] = good_df['nacef_5'].astype(str)
good_df['orgnr_n_1'] = good_df['orgnr_n_1'].astype(str)
good_df['b_kommunenr'] = good_df['b_kommunenr'].astype(str)
good_df['forbruk'] = good_df['forbruk'].astype(float)
good_df['salgsint'] = good_df['salgsint'].astype(float)
good_df['tmp_no_p4005'] = good_df['tmp_no_p4005'].astype(float)
good_df['foretak_omsetning'] = good_df['foretak_omsetning'].astype(float)

# good_df['geometry'] = good_df['geometry'].apply(lambda geom: geom.wkt if geom is not None else None)

good_df.head()

In [ ]:
good_df.to_parquet(
    f"gs://ssb-prod-noeku-data-produkt/temp/knn_varehandel.parquet",
    storage_options={"token": AuthClient.fetch_google_credentials()},
)

In [ ]:
fil_path = [
    f
    for f in fs.glob(
        f"gs://ssb-prod-noeku-data-produkt/temp/knn_varehandel.parquet"
    )
    if f.endswith(".parquet")
]

# Use the ParquetDataset to read multiple files
dataset = pq.ParquetDataset(fil_path, filesystem=fs)
table = dataset.read()

# Convert to Pandas DataFrame
good_df = table.to_pandas()

# count how many rows where tot_driftskost_fordelt is NaN
print(good_df['tot_driftskost_fordelt'].isnull().sum())

In [ ]:

condition = (good_df['foretak_omsetning'] == 0) | (good_df['foretak_driftskostnad'] == 0)

# Drop the rows that meet the condition
good_df = good_df[~condition]

good_df["gjeldende_driftsk_kr"] = pd.to_numeric(good_df["gjeldende_driftsk_kr"], errors="coerce")
good_df["b_sysselsetting_syss"] = pd.to_numeric(good_df["b_sysselsetting_syss"], errors="coerce")

good_df.drop(['tot_driftskost_fordelt'], axis=1, inplace=True)

good_df["driftsk"] = good_df["gjeldende_driftsk_kr"]

grouped = (
    good_df.groupby("id")[["driftsk"]].sum().reset_index()
)


grouped.rename(
    columns={"driftsk": "tot_driftskost_fordelt"},
    inplace=True,
)

good_df = pd.merge(good_df, grouped, on="id", how="left")

# Convert columns to numeric
good_df["tot_driftskost_fordelt"] = pd.to_numeric(
    good_df["tot_driftskost_fordelt"], errors="coerce"
)
good_df["driftsk"] = pd.to_numeric(good_df["driftsk"], errors="coerce")

good_df["b_sysselsetting_syss"] = pd.to_numeric(good_df["b_sysselsetting_syss"], errors="coerce")

# Convert 'lonn' to numeric, replacing comma with dot and handling errors
good_df["lonn"] = good_df["lonn"].str.replace(',', '.').astype(float)
good_df["lonn"] = good_df["lonn"] / 100

# Calculate drkost_share
# Calculate drkost_share
good_df["drkost_share"] = good_df.apply(
    lambda row: row["lonn"] if row["foretak_driftskostnad"] != 0 and row["tot_driftskost_fordelt"] == 0 else (row["driftsk"] / row["tot_driftskost_fordelt"] if row["tot_driftskost_fordelt"] != 0 else np.nan),
    axis=1
)

# Handle any NaN or inf values in drkost_share
good_df['drkost_share'].replace([np.inf, -np.inf], np.nan, inplace=True)
good_df['drkost_share'].fillna(0, inplace=True)

# Calculate total b_sysselsetting_syss per id
good_df['total_syss'] = good_df.groupby('id')['b_sysselsetting_syss'].transform('sum')

good_df["total_syss"] = pd.to_numeric(good_df["total_syss"], errors="coerce")

# Calculate the share of b_sysselsetting_syss per id
good_df['syss_share'] = good_df['b_sysselsetting_syss'] / good_df['total_syss']

# Update drkost_share for the specified conditions
good_df.loc[
    (good_df['tot_driftskost_fordelt'] == 0) & 
    (good_df['drkost_share'] == 0) & 
    (good_df['foretak_driftskostnad'] != 0), 
    'drkost_share'
] = good_df['syss_share']

# Round drkost_share to 10 decimal points
good_df['drkost_share'] = good_df['drkost_share'].round(10)


# Calculate new_drkost
good_df["new_drkost"] = good_df["drkost_share"] * good_df["foretak_driftskostnad"]

# Replace NaN in new_drkost with gjeldende_driftsk_kr
good_df["new_drkost"].fillna(good_df["gjeldende_driftsk_kr"], inplace=True)
good_df['new_drkost'] = good_df['new_drkost'].astype(float)

# if foretak_driftskostnad = 0 then new_drkost = 0
good_df.loc[good_df['foretak_driftskostnad'] == 0, 'new_drkost'] = 0

# if foretak_omsetning = 0 then new_oms = 0
good_df.loc[good_df['foretak_omsetning'] == 0, 'new_oms'] = 0

# Drop the unnecessary columns
# good_df.drop(['tot_driftskost_fordelt', 'drkost_share'], axis=1, inplace=True)

In [ ]:
#### TEST
test_grouped = (
    good_df.groupby("id")[["new_drkost"]].sum().reset_index()
)

test_grouped.rename(
    columns={"new_drkost": "test_tot_drkost_fordelt"},
    inplace=True,
)

test = pd.merge(good_df, test_grouped, on="id", how="left")

test['drkost_diff'] = test['foretak_driftskostnad'] - test['test_tot_drkost_fordelt']

test = test.sort_values(by='drkost_diff', ascending=True)

# change pd option to show all rows 
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)


test.head()

In [ ]:
grouped = (
    good_df.groupby("id")[["new_oms"]].sum().reset_index()
)

grouped.rename(
    columns={"new_oms": "tot_oms_fordelt"},
    inplace=True,
)

good_df = pd.merge(good_df, grouped, on="id", how="left")

mask_regtype_04 = good_df['regtype'] == '04'
mask_regtype_not_04 = good_df['regtype'] != '04'

# Step 2: Update new_oms where regtype is '04'
good_df.loc[mask_regtype_04, 'new_oms'] = good_df.loc[mask_regtype_04, 'new_drkost']

# Step 2: Group by 'id' and sum 'new_oms' where regtype is '04'
total_helper_oms = good_df.loc[mask_regtype_04].groupby('id')['new_oms'].sum().reset_index()



# Rename the aggregated 'new_oms' to 'new_oms_total_helper' for clarity
total_helper_oms.rename(columns={'new_oms': 'new_oms_total_helper'}, inplace=True)

# Step 3: Merge the aggregated result back into the original DataFrame
good_df = pd.merge(good_df, total_helper_oms, on='id', how='left', suffixes=('', '_total_helper'))

# Reset index to ensure it's unique
good_df.reset_index(drop=True, inplace=True)

# Convert 'foretak_omsetning' to numeric, setting errors to NaN
good_df['foretak_omsetning'] = pd.to_numeric(good_df['foretak_omsetning'], errors='coerce')

# Fill NaN values that might have been introduced by conversion errors
good_df['foretak_omsetning'].fillna(0, inplace=True)
good_df['new_oms_total_helper'].fillna(0, inplace=True)

good_df['new_oms_total_helper'] = pd.to_numeric(good_df['new_oms_total_helper'], errors='coerce')

# Step 4: Subtract 'new_oms_total_helper' from 'foretak_omsetning'
good_df['total_rest_oms'] = good_df['foretak_omsetning'] - good_df['new_oms_total_helper']

# Step 2: Group by 'id' and sum 'new_oms' where regtype is not '04'
total_non_helper_oms = good_df.loc[mask_regtype_not_04].groupby('id')['new_oms'].sum().reset_index()

# Step 3: Merge the aggregated result back into the original DataFrame
good_df = pd.merge(good_df, total_non_helper_oms, on='id', how='left', suffixes=('', '_total_non_helper'))

good_df['new_oms_total_non_helper'].fillna(0, inplace=True)



# test 4

# Convert 'new_oms' and 'new_oms_total_non_helper' to numeric, coercing errors to NaN
good_df['new_oms'] = pd.to_numeric(good_df['new_oms'], errors='coerce')
good_df['new_oms_total_non_helper'] = pd.to_numeric(good_df['new_oms_total_non_helper'], errors='coerce')

# convert Nan in new_oms_total_non_helper to 0
good_df['new_oms_total_non_helper'].fillna(0, inplace=True)

## Here we need to add some stuff
# Calculate total lonn per id excluding regtype '04'
good_df['total_lonn_non_04'] = good_df[mask_regtype_not_04].groupby('id')['lonn'].transform('sum')

# Ensure total_lonn_non_04 is numeric and handle any conversion issues
good_df['total_lonn_non_04'] = pd.to_numeric(good_df['total_lonn_non_04'], errors='coerce')

# Recalculate lonn for non-'04' rows to sum to 100% per id
good_df['lonn_non_04_share'] = np.where(
    mask_regtype_not_04,
    good_df['lonn'] / good_df['total_lonn_non_04'],
    0
)

good_df['lonn_non_04_share'].fillna(0, inplace=True)

# Calculate total b_sysselsetting_syss per id excluding regtype '04'
good_df['total_syss_non_04'] = good_df[mask_regtype_not_04].groupby('id')['b_sysselsetting_syss'].transform('sum')

# Calculate the share of b_sysselsetting_syss excluding regtype '04'
good_df['syss_share_non_04'] = good_df['b_sysselsetting_syss'] / good_df['total_syss_non_04']

good_df['syss_share_non_04'].fillna(0, inplace=True)

# Calculate oms_share_non_helpers with the new condition
good_df['oms_share_non_helpers'] = good_df.apply(
    lambda row: 1 if row['total_rest_oms'] == 0 else (
        row['lonn_non_04_share'] if row['foretak_omsetning'] != 0 and row['tot_oms_fordelt'] == 0 else (
            row["new_oms"] / row["new_oms_total_non_helper"] if row["new_oms_total_non_helper"] != 0 else row['syss_share_non_04'])),
    axis=1
)


######################################

# Handle any NaN or inf values in oms_share_non_helpers
good_df['oms_share_non_helpers'].replace([np.inf, -np.inf], np.nan, inplace=True)
good_df['oms_share_non_helpers'].fillna(0, inplace=True)

# Ensure oms_share_non_helpers is set to 0 where regtype is '04'
good_df.loc[good_df['regtype'] == '04', 'oms_share_non_helpers'] = 0

# Finally, update new_oms with the calculated oms_share_non_helpers
good_df['new_oms'] = np.where(good_df['regtype'] == '04', 
                              good_df['new_oms'], 
                              good_df['oms_share_non_helpers'] * good_df['total_rest_oms'])

# Drop any temporary or helper columns if necessary
# good_df.drop(['total_lonn_non_04', 'total_syss_non_04', 'lonn_non_04_share'], axis=1, inplace=True, errors='ignore')

## end new code

# old code
# good_df['oms_share_non_helpers'] = good_df['new_oms'] / good_df['new_oms_total_non_helper']

# # Set the oms_share_non_helpers to 0 where regtype is '04'
# good_df.loc[good_df['regtype'] == '04', 'oms_share_non_helpers'] = 0

# # Replace NaN in 'oms_share_non_helpers' resulting from division by zero or NaN in the denominator
# good_df['oms_share_non_helpers'].fillna(0, inplace=True)

# # Optionally, replace NaN resulting from any remaining errors with a default value
# good_df.fillna(0, inplace=True)

# # Step 1: Use numpy.where to conditionally update new_oms
# good_df['new_oms'] = np.where(good_df['regtype'] == '04', 
#                               good_df['new_oms'], 
#                               good_df['oms_share_non_helpers'] * good_df['total_rest_oms'])

In [ ]:
##### TEST

test_grouped = (
    good_df.groupby("id")[["new_oms"]].sum().reset_index()
)

test_grouped.rename(
    columns={"new_oms": "test_tot_oms_fordelt"},
    inplace=True,
)

test = pd.merge(good_df, test_grouped, on="id", how="left")

test['oms_diff'] = test['foretak_omsetning'] - test['test_tot_oms_fordelt']

test = test.sort_values(by='oms_diff', ascending=False)

# change pd option to show all rows 
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)


test.head()



In [ ]:
# 1
# Define the values for w_naring_vh, w_nace1_ikke_vh, and w_nace2_ikke_vh
w_naring_vh = ("45", "46", "47")
w_nace1_ikke_vh = "45.403"
w_nace2_ikke_vh = ("45.2", "46.1")

enhetene_brukes = good_df.copy()

# Filter the DataFrame based on conditions and create vhbed variable
enhetene_brukes["vhbed"] = 0

# Check if the first two characters of 'naring' are in w_naring_vh
enhetene_brukes.loc[
    enhetene_brukes["tmp_sn2007_5"].str[:2].isin(w_naring_vh), "vhbed"
] = 1

# Check if 'naring' is in w_nace1_ikke_vh
enhetene_brukes.loc[enhetene_brukes["tmp_sn2007_5"] == w_nace1_ikke_vh, "vhbed"] = 0

# Check if the first four characters of 'naring' are in w_nace2_ikke_vh
enhetene_brukes.loc[
    enhetene_brukes["tmp_sn2007_5"].str[:4].isin(w_nace2_ikke_vh), "vhbed"
] = 0

enhetene_brukes = enhetene_brukes.drop_duplicates(subset=["orgnr_n_1", "lopenr", "radnr", "v_orgnr"])

In [ ]:
aar = 2021

fil_path = [
    f
    for f in fs.glob(
        f"gs://ssb-prod-noeku-data-produkt/statistikkfiler/g{aar}/statistikkfil_foretak_nr.parquet"
    )
    if f.endswith(".parquet")
]

# Use the ParquetDataset to read multiple files
dataset = pq.ParquetDataset(fil_path, filesystem=fs)
table = dataset.read()

# Convert to Pandas DataFrame
foretak = table.to_pandas()



foretak = foretak[["enhets_id",'orgnr_foretak','nopost_p4295','nopost_p4295','nopost_p4995','nopost_p6200', 'naring_f', 'reg_type', 'nopost_p4005']]

# filter for where substr first 2 characters of naring_f is in 45, 46 or 47
foretak = foretak[foretak['naring_f'].str[:2].isin(['45', '46', '47'])]

# rename columns
foretak.rename(
    columns={
        "naring_f": "foretaksnaring",
        "enhets_id": "id",
        "nopost_p4295": "no4295_f",
        "nopost_p4995": "no4995_f",
        "nopost_p6200": "no6200_f",
        "nopost_p4005": "forbruk_f",
    },
    inplace=True,
)

foretak = foretak[["id", "forbruk_f"]]

# drop duplicate orgnr_foretak and reset index
foretak = foretak.drop_duplicates(subset=["id"]).reset_index(drop=True)

# # Display the resulting DataFrame
foretak.head()

In [ ]:
enhetene_brukes = enhetene_brukes.merge(foretak, on="id", how="left")

In [ ]:
enhetene_brukes ['check'] = enhetene_brukes["foretak_driftskostnad"] - enhetene_brukes["forbruk"]

# if check is negative, then 'forbruk' = 'forbruk' / 1000
enhetene_brukes ['forbruk'] = np.where(
    enhetene_brukes['check'] < 0, enhetene_brukes['forbruk'] / 1000, enhetene_brukes['forbruk']
)
enhetene_brukes ['check'] = enhetene_brukes["foretak_driftskostnad"] - enhetene_brukes["forbruk"]
# print out the count number of rows where check is negative
print(enhetene_brukes[enhetene_brukes['check'] < 0].shape[0])

In [ ]:
enhetene_brukes['forbruk'] = np.where(
    enhetene_brukes['check'] < 0, enhetene_brukes['forbruk_f'], enhetene_brukes['forbruk']
)
enhetene_brukes ['check'] = enhetene_brukes["foretak_driftskostnad"] - enhetene_brukes["forbruk"]
# print out the count number of rows where check is negative
print(enhetene_brukes[enhetene_brukes['check'] < 0].shape[0])

In [ ]:
enhetene_brukes.head()

check = enhetene_brukes[enhetene_brukes['id'] == '11520914']
check.head()
70231
2251296

In [ ]:
# 2
salgsint_forbruk = enhetene_brukes[
    [
        "orgnr_n_1",
        "lopenr",
        "v_orgnr",
        "forbruk",
        "salgsint",
        "radnr",
        "nacef_5",
        "tmp_sn2007_5",
        "new_oms",
        "vhbed",
    ]
]

salgsint_forbruk.shape

In [ ]:
# 3

har = salgsint_forbruk[salgsint_forbruk.groupby("orgnr_n_1")["vhbed"].transform("any")]
# Extract the 'orgnr_n_1' column
har = har[["orgnr_n_1"]]

# Remove duplicates
har.drop_duplicates(inplace=True)

har.shape

In [ ]:
# 4

ikke_har = salgsint_forbruk[
    ~salgsint_forbruk.groupby("orgnr_n_1")["vhbed"].transform("any")
]
ikke_har = ikke_har[["orgnr_n_1"]]
ikke_har.drop_duplicates(inplace=True)

ikke_har["ikkevbed"] = 1

ikke_har.shape

In [ ]:
# 5


# Merge ikke_har into salgsint_forbruk with a left join on the 'id' column
salgsint_forbruk_update1 = pd.merge(
    salgsint_forbruk, ikke_har, on="orgnr_n_1", how="left"
)

# salgsint_forbruk_update1['ikkevbed'].fillna(0, inplace=True)

# Update 'vhbed' to 1 where 'ikkevbed' is 1
salgsint_forbruk_update1.loc[salgsint_forbruk_update1["ikkevbed"] == 1, "vhbed"] = 1

salgsint_forbruk_update1.shape

In [ ]:
# 6
# Assuming your original DataFrame is named salgsint_forbruk_update1
# Replace 'new_oms', 'orgnr_foretak', 'lopnr', 'vhbed' with the actual column names in your DataFrame

# Create sum1 DataFrame for vhbed=1
sum1 = (
    salgsint_forbruk_update1[salgsint_forbruk_update1["vhbed"] == 1]
    .groupby(["orgnr_n_1", "lopenr"])["new_oms"]
    .sum()
    .reset_index()
)
sum1.rename(columns={"new_oms": "sumoms_vh"}, inplace=True)

# Create sum2 DataFrame for vhbed=0
sum2 = (
    salgsint_forbruk_update1[salgsint_forbruk_update1["vhbed"] == 0]
    .groupby(["orgnr_n_1", "lopenr"])["new_oms"]
    .sum()
    .reset_index()
)
sum2.rename(columns={"new_oms": "sumoms_andre"}, inplace=True)

In [ ]:
salgsint_forbruk_update1.head()

In [ ]:
# Perform an outer join of sum1 and sum2 by 'orgnr_foretak' and 'lopnr'
sum3 = pd.merge(sum1, sum2, on=["orgnr_n_1", "lopenr"], how="outer")
sum3.shape

In [ ]:
# 8

salgsint_forbruk_update2 = pd.merge(
    salgsint_forbruk_update1, sum3, on=["orgnr_n_1", "lopenr"], how="outer"
)
salgsint_forbruk_update2.shape

In [ ]:
# 9 & 10
# Assuming 'orgnr_n_1', 'lopenr', and 'rad_nr' are the actual column names
# Replace them with the actual names in your DataFrame

# Sort the DataFrame by 'orgnr_n_1', 'lopenr', and 'rad_nr'
salgsint_forbruk_update2.sort_values(by=["orgnr_n_1", "lopenr", "radnr"], inplace=True)

salgsint_forbruk_update2.sort_values(by=["orgnr_n_1", "lopenr", "vhbed"], inplace=True)

# Display the sorted DataFrame
salgsint_forbruk_update2.shape

In [ ]:
# 11

# Assuming your DataFrame is named salgsint_forbruk
# Replace 'orgnr_foretak', 'lopenr', 'vhbed', 'vhf' with the actual column names

# Sort the DataFrame by 'orgnr_foretak' and 'lopenr'

salgsint_forbruk_update3 = salgsint_forbruk_update2.copy()

salgsint_forbruk_update3.sort_values(by=["orgnr_n_1", "lopenr"], inplace=True)

# Create a new variable 'vhf' based on the values of 'vhbed'
salgsint_forbruk_update3["vhf"] = salgsint_forbruk_update3.groupby(
    ["orgnr_n_1", "lopenr"]
)["vhbed"].transform("first")

# Retain the value of 'vhf' from the first observation in each group
salgsint_forbruk_update3["vhf"] = salgsint_forbruk_update3.groupby(
    ["orgnr_n_1", "lopenr"]
)["vhf"].transform("first")

# Apply labels to the variables
salgsint_forbruk_update3["vhbed"] = salgsint_forbruk_update3["vhbed"].astype(str)
salgsint_forbruk_update3["vhf"] = salgsint_forbruk_update3["vhf"].astype(str)

label_map_vhbed = {"1": "varehandelsbedrift", "0": "annen type bedrift"}
label_map_vhf = {
    "1": "foretaket har kun varehandelsbedrifter eller ingen",
    "0": "har varehandel og annen bedrift (blandingsnæringer)",
}

salgsint_forbruk_update3["vhbed"] = salgsint_forbruk_update3["vhbed"].map(
    label_map_vhbed
)
salgsint_forbruk_update3["vhf"] = salgsint_forbruk_update3["vhf"].map(label_map_vhf)

salgsint_forbruk_update3.shape

In [ ]:
# 11

# Assuming your DataFrame is named salgsint_forbruk
# Replace 'vhf' with the actual name of your column

# Filter rows where vhf is 'foretaket har kun varehandelsbedrifter eller ingen'
vhf_condition = (
    salgsint_forbruk_update3["vhf"]
    == "foretaket har kun varehandelsbedrifter eller ingen"
)
vhf_df = salgsint_forbruk_update3.loc[vhf_condition]

# Filter rows where vhf is not 'foretaket har kun varehandelsbedrifter eller ingen'
andre_df = salgsint_forbruk_update3.loc[~vhf_condition]

print(vhf_df.shape)
print(andre_df.shape)

In [ ]:
# 12

vhf_df["nokkel"] = vhf_df["new_oms"] / vhf_df["sumoms_vh"]

# Convert 'salgsint' column to numeric
vhf_df["salgsint"] = pd.to_numeric(vhf_df["salgsint"], errors="coerce")
vhf_df["forbruk"] = pd.to_numeric(vhf_df["forbruk"], errors="coerce")


vhf_df["bedr_salgsint"] = round(vhf_df["salgsint"] * vhf_df["nokkel"])
vhf_df["bedr_forbruk"] = round(vhf_df["forbruk"] * vhf_df["nokkel"])

vhf_df.shape

In [ ]:
# 13

andre_df["forbruk"] = pd.to_numeric(andre_df["forbruk"], errors="coerce")
andre_df["salgsint"] = pd.to_numeric(andre_df["salgsint"], errors="coerce")


# Assuming 'andre' is your DataFrame
andre_df["avanse"] = andre_df["forbruk"] / andre_df["salgsint"]

# Filter rows where vhbed is 1
vh_bedriftene = andre_df[andre_df["vhbed"] == "varehandelsbedrift"].copy()

# Calculate 'nokkel', 'bedr_salgsint', and 'bedr_forbruk' for vh-bedriftene
vh_bedriftene["nokkel"] = vh_bedriftene["new_oms"] / vh_bedriftene["sumoms_vh"]
vh_bedriftene["bedr_salgsint"] = round(
    vh_bedriftene["salgsint"] * vh_bedriftene["nokkel"]
)
vh_bedriftene.loc[
    vh_bedriftene["bedr_salgsint"] > vh_bedriftene["new_oms"], "bedr_salgsint"
] = vh_bedriftene["new_oms"]
vh_bedriftene["bedr_forbruk"] = round(
    vh_bedriftene["bedr_salgsint"] * vh_bedriftene["avanse"]
)

# Summarize vh-bedriftene
brukt1 = (
    vh_bedriftene.groupby(["orgnr_n_1", "lopenr"])
    .agg({"bedr_salgsint": "sum", "bedr_forbruk": "sum"})
    .reset_index()
)

# Merge summarized values back to 'andre'
andre = pd.merge(andre_df, brukt1, on=["orgnr_n_1", "lopenr"], how="left")

# Calculate 'resten1' and 'resten2'
andre["resten1"] = andre["salgsint"] - andre["bedr_salgsint"]
andre["resten2"] = andre["forbruk"] - andre["bedr_forbruk"]

# Filter rows where vhbed is not 1
blanding_av_vh_og_andre = andre[andre["vhbed"] != "varehandelsbedrift"].copy()

# Calculate 'nokkel', 'bedr_salgsint', and 'bedr_forbruk' for blending of vh and other industries
blanding_av_vh_og_andre["nokkel"] = (
    blanding_av_vh_og_andre["new_oms"] / blanding_av_vh_og_andre["sumoms_andre"]
)
blanding_av_vh_og_andre["bedr_salgsint"] = round(
    blanding_av_vh_og_andre["resten1"] * blanding_av_vh_og_andre["nokkel"]
)
blanding_av_vh_og_andre["bedr_forbruk"] = round(
    blanding_av_vh_og_andre["resten2"] * blanding_av_vh_og_andre["nokkel"]
)

# Combine the two subsets back into 'andre'
andre = pd.concat([vh_bedriftene, blanding_av_vh_og_andre], ignore_index=True)

andre.sort_values(by=["orgnr_n_1", "lopenr"], inplace=True)

oppdatere_hv = pd.concat([vhf_df, andre], ignore_index=True)

oppdatere_hv = oppdatere_hv[
    ["orgnr_n_1", "lopenr", "radnr", "bedr_forbruk", "bedr_salgsint"]
]

oppdatere_hv.shape

In [ ]:
enhetene_brukes2 = pd.merge(
    enhetene_brukes, oppdatere_hv, on=["orgnr_n_1", "lopenr", "radnr"]
)

enhetene_brukes2.shape

In [ ]:
# Step 1: Identify IDs that appear more than once
duplicate_ids = enhetene_brukes2['id'][enhetene_brukes2['id'].duplicated(keep=False)]

# Step 2: Update regtype to '02' where regtype is '01' and the id appears more than once
enhetene_brukes2.loc[(enhetene_brukes2['regtype'] == '01') & (enhetene_brukes2['id'].isin(duplicate_ids)), 'regtype'] = '02'

enhetene_brukes2.loc[enhetene_brukes2['regtype'] == '01', 'new_oms'] = enhetene_brukes2['foretak_omsetning']
enhetene_brukes2.loc[enhetene_brukes2['regtype'] == '01', 'new_drkost'] = enhetene_brukes2['foretak_driftskostnad']
enhetene_brukes2.loc[enhetene_brukes2['regtype'] == '01', 'bedr_salgsint'] = enhetene_brukes2['salgsint']
enhetene_brukes2.loc[enhetene_brukes2['regtype'] == '01', 'bedr_forbruk'] = enhetene_brukes2['forbruk']

In [ ]:
enhetene_brukes2.head()

In [ ]:
#### TEST
test_grouped = (
    enhetene_brukes2.groupby("id")[["new_drkost"]].sum().reset_index()
)

test_grouped.rename(
    columns={"new_drkost": "test_tot_drkost_fordelt"},
    inplace=True,
)

test1 = pd.merge(enhetene_brukes2, test_grouped, on="id", how="left")

test1['drkost_diff'] = test1['foretak_driftskostnad'] - test1['test_tot_drkost_fordelt']

test1 = test1.sort_values(by='drkost_diff', ascending=True)

In [ ]:
test1.head()

In [ ]:
test_grouped = (
    enhetene_brukes2.groupby("id")[["new_oms"]].sum().reset_index()
)

test_grouped.rename(
    columns={"new_oms": "test_tot_oms_fordelt"},
    inplace=True,
)

test2 = pd.merge(enhetene_brukes2, test_grouped, on="id", how="left")

In [ ]:
test2.head()

In [ ]:
test2['oms_diff'] = test2['foretak_omsetning'] - test2['test_tot_oms_fordelt']

test2 = test2.sort_values(by='oms_diff', ascending=True)

# change pd option to show all rows 
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

In [ ]:
test1.head()

In [ ]:
test2.head()

In [ ]:
rettes = enhetene_brukes2.copy()
rettes["new_drkost"].isna().sum()
rettes["oms"] = rettes["new_oms"]
rettes["driftsk"] = rettes["gjeldende_driftsk_kr"]

In [ ]:
# Convert columns to numeric
rettes["tot_driftskost_fordelt"] = pd.to_numeric(
    rettes["tot_driftskost_fordelt"], errors="coerce"
)
rettes["driftsk"] = pd.to_numeric(rettes["driftsk"], errors="coerce")

 


# rettes["drkost_share"] = rettes["driftsk"] / rettes["tot_driftskost_fordelt"]

In [ ]:
# rettes["drkost_share"].isna().sum()

In [ ]:
# rettes["new_drkost"] = rettes["drkost_share"] * rettes["foretak_driftskostnad"]
# rettes["new_drkost"] = rettes["new_drkost"].replace([np.inf, -np.inf], np.nan)

# rettes["profit_ratio"] = rettes["foretak_driftskostnad"] / rettes["foretak_omsetning"]

# # if tot_driftskost_fordelt = 0 and 'new_drkost' is NaN then new_drkost = "profit_ratio" * new_oms
# rettes["new_drkost"] = np.where(
#     (rettes["tot_driftskost_fordelt"] == 0) & (rettes["new_drkost"].isna()),
#     rettes["profit_ratio"] * rettes["new_oms"],
#     rettes["new_drkost"],
# )

# rettes["new_drkost"].isna().sum()

# # set "new_drkost" nan to 0
# rettes["new_drkost"] = rettes["new_drkost"].fillna(0)
# rettes["new_drkost"].isna().sum()

In [ ]:
#### TEST RETTES

#### TEST
test_grouped = (
    rettes.groupby("id")[["new_drkost"]].sum().reset_index()
)

test_grouped.rename(
    columns={"new_drkost": "test_tot_drkost_fordelt"},
    inplace=True,
)

test1 = pd.merge(rettes, test_grouped, on="id", how="left")

test1['drkost_diff'] = test1['foretak_driftskostnad'] - test1['test_tot_drkost_fordelt']

test1 = test1.sort_values(by='drkost_diff', ascending=True)

test_grouped = (
    rettes.groupby("id")[["new_oms"]].sum().reset_index()
)

test_grouped.rename(
    columns={"new_oms": "test_tot_oms_fordelt"},
    inplace=True,
)

test2 = pd.merge(rettes, test_grouped, on="id", how="left")

test2['oms_diff'] = test2['foretak_omsetning'] - test2['test_tot_oms_fordelt']

test2 = test2.sort_values(by='oms_diff', ascending=True)

# change pd option to show all rows 
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

In [ ]:
test1.head()

In [ ]:
test2.head()

# Kontrol Drkost for forbuk og lønn. 

In [ ]:
rettes2 = rettes.copy()



rettes2["drkost_temp"] = rettes2["new_drkost"]

# Fill NaN in 'drkost_temp' with 0
rettes2["drkost_temp"] = rettes2["drkost_temp"].fillna(0)



# replace comma with a '.' for gjeldende_lonn_kr
rettes2["gjeldende_lonn_kr"] = rettes2["gjeldende_lonn_kr"].str.replace(",", ".")

rettes2["gjeldende_lonn_kr"] = pd.to_numeric(
    rettes2["gjeldende_lonn_kr"], errors="coerce"
).fillna(0)

# rettes2["bedr_forbruk"] = rettes2["bedr_forbruk"].str.replace(",", ".")
rettes2["bedr_forbruk"] = pd.to_numeric(
    rettes2["bedr_forbruk"], errors="coerce"
).fillna(0)


rettes2["lonn_+_forbruk"] = rettes2["gjeldende_lonn_kr"] + rettes2["bedr_forbruk"]



# Perform the if operation
condition = rettes2["drkost_temp"] < rettes2["lonn_+_forbruk"]
rettes2["drkost_temp"] = np.where(
    condition, rettes2["lonn_+_forbruk"], rettes2["drkost_temp"]
)
rettes2["theif"] = np.where(condition, 1, 0)

rettes2.shape

In [ ]:
# Filter the DataFrame
dkvars = rettes2[rettes2.groupby("orgnr_n_1")["theif"].transform("any")]

dkvars.head()

In [ ]:
# Calculate 'utskudd'
dkvars["utskudd"] = (
    dkvars["new_drkost"] - dkvars["gjeldende_lonn_kr"] - dkvars["bedr_forbruk"]
)
dkvars["utskudd"] = abs(dkvars["utskudd"])

# Keep selected columns
columns_to_keep = [
    "orgnr_n_1",
    "lopenr",
    "radnr",
    "utskudd",
    "new_drkost",
    "drkost_temp",
    "theif",
    "gjeldende_lonn_kr",
    "bedr_forbruk",
]
dkvars = dkvars[columns_to_keep]

# Display the resulting DataFrame
dkvars.shape

In [ ]:
# Calculate sum of 'utskudd' grouped by 'orgnr_foretak', 'lopenr', and 'tyv'
sum7b = dkvars.groupby(["orgnr_n_1", "lopenr", "theif"])["utskudd"].sum().reset_index()

# Display the result
sum7b.head()

In [ ]:
# Transpose the result
sum7b_transposed = sum7b.pivot(
    index=["orgnr_n_1", "lopenr"], columns="theif", values="utskudd"
).reset_index()

# Rename columns as per SAS code
sum7b_transposed.rename(columns={0: "thief0", 1: "thief1"}, inplace=True)

sum7b_transposed = sum7b_transposed[["orgnr_n_1", "lopenr", "thief0", "thief1"]]

# Display the transposed result
sum7b_transposed.head()

In [ ]:
# merge sums
dkvars_2 = pd.merge(dkvars, sum7b_transposed, on=["orgnr_n_1", "lopenr"], how="inner")
# dkvars_2.head()

In [ ]:
# Apply conditional logic
pd.set_option("display.float_format", "{:.2f}".format)
dkvars_2["andel1"] = np.where(
    dkvars_2["theif"] == 0, dkvars_2["utskudd"] / dkvars_2["thief0"], np.nan
)
dkvars_2["andel2"] = np.where(
    dkvars_2["theif"] == 0, np.round(dkvars_2["andel1"] * dkvars_2["thief1"]), np.nan
)
# dkvars_2['new_drkost'] = np.where(dkvars_2['theif'] == 0, np.sum(dkvars_2['drkost_temp'] - dkvars_2['andel2'], axis=0), dkvars_2['drkost_temp'])
dkvars_2["new_drkost"] = np.where(
    dkvars_2["theif"] == 0,
    dkvars_2["drkost_temp"] - dkvars_2["andel2"],
    dkvars_2["drkost_temp"],
)

# Keep selected columns
columns_to_keep = ["orgnr_n_1", "lopenr", "radnr", "new_drkost"]
dkvars_3 = dkvars_2[columns_to_keep]

# dkvars_2.head(50)

good_final = dkvars_2.copy()

In [ ]:
dkvars_3.head()

In [ ]:
merged_df = pd.merge(rettes2, dkvars_3, how='left', left_on=['orgnr_n_1', 'lopenr', 'radnr'], right_on=['orgnr_n_1', 'lopenr', 'radnr'], suffixes=('', '_updated'))

# # Step 1: Identify IDs that appear more than once
# duplicate_ids = merged_df['id'][merged_df['id'].duplicated(keep=False)]

# # Step 2: Update regtype to '02' where regtype is '01' and the id appears more than once
# merged_df.loc[(merged_df['regtype'] == '01') & (merged_df['id'].isin(duplicate_ids)), 'regtype'] = '02'

# merged_df.loc[merged_df['regtype'] == '01', 'new_oms'] = merged_df['foretak_omsetning']
# merged_df.loc[merged_df['regtype'] == '01', 'new_drkost'] = merged_df['foretak_driftskostnad']
# merged_df.loc[merged_df['regtype'] == '01', 'bedr_salgsint'] = merged_df['salgsint']
# merged_df.loc[merged_df['regtype'] == '01', 'bedr_forbruk'] = merged_df['forbruk']

In [ ]:
merged_df['new_drkost'] = merged_df['new_drkost_updated'].combine_first(merged_df['new_drkost'])

duplicate_ids = merged_df['id'][merged_df['id'].duplicated(keep=False)]

# Step 2: Update regtype to '02' where regtype is '01' and the id appears more than once
merged_df.loc[(merged_df['regtype'] == '01') & (merged_df['id'].isin(duplicate_ids)), 'regtype'] = '02'

merged_df.loc[enhetene_brukes2['regtype'] == '01', 'new_oms'] = merged_df['foretak_omsetning']
merged_df.loc[merged_df['regtype'] == '01', 'new_drkost'] = merged_df['foretak_driftskostnad']
merged_df.loc[merged_df['regtype'] == '01', 'bedr_salgsint'] = merged_df['salgsint']
merged_df.loc[merged_df['regtype'] == '01', 'bedr_forbruk'] = merged_df['forbruk']

test_grouped = (
    merged_df.groupby("id")[["new_drkost"]].sum().reset_index()
)

test_grouped.rename(
    columns={"new_drkost": "test_tot_drkost_fordelt"},
    inplace=True,
)

temp = pd.merge(merged_df, test_grouped, on="id", how="left")

temp['drkost_diff'] = temp['foretak_driftskostnad'] - temp['test_tot_drkost_fordelt']

temp = temp.sort_values(by='drkost_diff', ascending=True)

mask = temp['drkost_diff'].abs() <= 1000

# Create a new DataFrame with the rows to be excluded
check_manually = merged_df[~mask]

# Update the original DataFrame to keep only the rows where the absolute value of 'drkost_diff' is <= 1000
merged_df = merged_df[mask]

In [ ]:
#### TEST RETTES

#### TEST
test_grouped = (
    merged_df.groupby("id")[["new_drkost"]].sum().reset_index()
)

test_grouped.rename(
    columns={"new_drkost": "test_tot_drkost_fordelt"},
    inplace=True,
)

test1 = pd.merge(merged_df, test_grouped, on="id", how="left")

test1['drkost_diff'] = test1['foretak_driftskostnad'] - test1['test_tot_drkost_fordelt']

test1 = test1.sort_values(by='drkost_diff', ascending=False)

test_grouped = (
    merged_df.groupby("id")[["new_oms"]].sum().reset_index()
)

test_grouped.rename(
    columns={"new_oms": "test_tot_oms_fordelt"},
    inplace=True,
)

test2 = pd.merge(merged_df, test_grouped, on="id", how="left")

test2['oms_diff'] = test2['foretak_omsetning'] - test2['test_tot_oms_fordelt']

test2 = test2.sort_values(by='oms_diff', ascending=True)

# change pd option to show all rows 
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)



In [ ]:
test1.head()

# Start of control for industry variables

In [ ]:
aar = 2021

fil_path = [
    f
    for f in fs.glob(
        f"gs://ssb-prod-noeku-data-produkt/statistikkfiler/g{aar}/statistikkfil_bedrifter_nr.parquet"
    )
    if f.endswith(".parquet")
]

# Use the ParquetDataset to read multiple files
dataset = pq.ParquetDataset(fil_path, filesystem=fs)
table = dataset.read()

# Convert to Pandas DataFrame
industri = table.to_pandas()



industri = industri[['enhets_id','orgnr_bedrift','nopost_p4295','nopost_p4995','nopost_p6200', 'naring_f']]

# filter for where substr first 2 characters of naring_f is in 45, 46 or 47
industri = industri[industri['naring_f'].str[:2].isin(['45', '46', '47'])]

# rename columns
industri.rename(columns={'enhets_id': 'id', 'orgnr_bedrift': 'v_orgnr'}, inplace=True)

industri.head()

In [ ]:
print(merged_df.shape)
print(industri.shape)

# merge with left join on id and v_orgnr
enhetene_brukes7b  = pd.merge(merged_df, industri, how='left', on=['id', 'v_orgnr'])
print(enhetene_brukes7b.shape)

# fill nan for bedr_forbruk, bedr_salgsint, nopost_p4295, nopost_p4995, nopost_p6200
enhetene_brukes7b ['bedr_forbruk'] = enhetene_brukes7b ['bedr_forbruk'].fillna(0)
enhetene_brukes7b ['bedr_salgsint'] = enhetene_brukes7b ['bedr_salgsint'].fillna(0)
enhetene_brukes7b ['nopost_p4295'] = enhetene_brukes7b ['nopost_p4295'].fillna(0)
enhetene_brukes7b ['nopost_p4995'] = enhetene_brukes7b ['nopost_p4995'].fillna(0)
enhetene_brukes7b ['nopost_p6200'] = enhetene_brukes7b ['nopost_p6200'].fillna(0)

#fill 

In [ ]:
enhetene_brukes7c = enhetene_brukes7b.copy()
enhetene_brukes7c.head()

In [ ]:
# fill new_drkost_updated NaN with new_drkost
# enhetene_brukes7c["new_drkost_updated"] = enhetene_brukes7c[
#     "new_drkost_updated"
# ].fillna(enhetene_brukes7c["new_drkost"])

# rename
enhetene_brukes7c.rename(
    columns={
        "orgnr_n_1": "orgnr_foretak",
        "new_drkost_updated": "tmp_ny_driftsk",
        "tmp_sn2007_5": "naring",
        "nacef_5": "foretaksnaring",
        "nopost_p4295": "no4295",
        "nopost_p4995": "no4995",
        "nopost_p6200": "no6200",
    },
    inplace=True,
)

industrivars = enhetene_brukes7c[["id","orgnr_foretak", "lopenr", "radnr", "tmp_ny_driftsk", "naring", "foretaksnaring", "no4295", "no4995", "no6200", 'foretak_driftskostnad', 'gjeldende_lonn_kr', 'bedr_forbruk']]

# sort by orgnr_foretak, lopenr, radnr
industrivars = industrivars.sort_values(by=["orgnr_foretak", "lopenr", "radnr"])

industrivars.head()

In [ ]:
# industrivars['gjeldende_lonn_kr'].value_counts()

In [ ]:
# n2 = substring first two charaters of naring
# fill naring NaN with foretaksnaring
industrivars["naring"] = industrivars["naring"].fillna(industrivars["foretaksnaring"])
industrivars["n2"] = industrivars["naring"].str[:2]

# change n2 to int"
industrivars["n2"] = industrivars["n2"].astype(int)

# create new variable called _industribedrift which shall = 1 if n2 greater than or equal to 05 or less than or equal to 33
industrivars["_industribedrift"] = np.where(
    (industrivars["n2"] >= 5) & (industrivars["n2"] <= 33), 1, 0
)

# Filter the DataFrame where _industribedrift is 1
filtered_df = industrivars[industrivars['_industribedrift'] == 1]

# Select distinct rows based on orgnr_foretak, lopenr, and _industribedrift
distinct_df = filtered_df[['orgnr_foretak', 'lopenr', '_industribedrift']].drop_duplicates()

# Rename the _industribedrift column to _har_indr
distinct_df = distinct_df.rename(columns={'_industribedrift': '_har_indr'})

# Sort the DataFrame by orgnr_foretak and lopenr
indrbed = distinct_df.sort_values(by=['orgnr_foretak', 'lopenr'])

del filtered_df, distinct_df

indrbed.head()

In [ ]:
industrivars = pd.merge(industrivars, indrbed, on=['orgnr_foretak', 'lopenr'], how='left', indicator=True)

# Filter the rows to include only those present in 'industrivars' or in both 'industrivars' and 'indrbed'
industrivars = industrivars[(industrivars['_merge'] == 'both') | (industrivars['_merge'] == 'left_only')]

# Drop the '_merge' column
industrivars = industrivars.drop(columns=['_merge'])

# sort by orgnr_foretak, lopenr, radnr
industrivars = industrivars.sort_values(by=["orgnr_foretak", "lopenr", "radnr"])

industrivars = industrivars.fillna(0)


industrivars.head()

In [ ]:
# Summing for industribedriftebne
sum101 = industrivars[industrivars['_industribedrift'] == 1].groupby(['orgnr_foretak', 'lopenr'])['tmp_ny_driftsk'].sum().reset_index()
sum101 = sum101.rename(columns={'tmp_ny_driftsk': 'fdk_indr'})

# Summing for vanlige bedrifter
sum102 = industrivars[industrivars['_industribedrift'] == 0].groupby(['orgnr_foretak', 'lopenr'])['tmp_ny_driftsk'].sum().reset_index()
sum102 = sum102.rename(columns={'tmp_ny_driftsk': 'fdk_vanl'})

# Joining sum101 with industrivars
industrivars = pd.merge(industrivars, sum101, on=['orgnr_foretak', 'lopenr'], how='left')

# Joining sum102 with industrivars
industrivars = pd.merge(industrivars, sum102, on=['orgnr_foretak', 'lopenr'], how='left')

# Sorting the final DataFrame
industrivars = industrivars.sort_values(by=['orgnr_foretak', 'lopenr', 'radnr'])

# fill Nan for _har_indr, fdk_indr, fdk_vanl with 0
industrivars = industrivars.fillna(0)

industrivars.head()


In [ ]:
aar = 2021

fil_path = [
    f
    for f in fs.glob(
        f"gs://ssb-prod-noeku-data-produkt/statistikkfiler/g{aar}/statistikkfil_foretak_nr.parquet"
    )
    if f.endswith(".parquet")
]

# Use the ParquetDataset to read multiple files
dataset = pq.ParquetDataset(fil_path, filesystem=fs)
table = dataset.read()

# Convert to Pandas DataFrame
foretak = table.to_pandas()



foretak = foretak[["enhets_id",'orgnr_foretak','nopost_p4005','nopost_p4295','nopost_p4995','nopost_p6200', 'naring_f', 'reg_type']]

# filter for where substr first 2 characters of naring_f is in 45, 46 or 47
foretak = foretak[foretak['naring_f'].str[:2].isin(['45', '46', '47'])]

# rename columns
foretak.rename(
    columns={
        "naring_f": "foretaksnaring",
        "enhets_id": "id",
        "nopost_p4295": "no4295_f",
        "nopost_p4995": "no4995_f",
        "nopost_p6200": "no6200_f",
        "nopost_p4005": "forbruk_f"
    },
    inplace=True,
)

foretak = foretak[["id", "no4295_f", "no4995_f", "no6200_f"]]

# drop duplicate orgnr_foretak and reset index
foretak = foretak.drop_duplicates(subset=["id"]).reset_index(drop=True)

# # Display the resulting DataFrame
foretak.head()

In [ ]:
# Check the data types of orgnr_foretak in both DataFrames
# print(foretak['orgnr_foretak'].dtype)
# print(industrivars['orgnr_foretak'].dtype)

# # Ensure they are the same type, if not, convert them to string
# foretak['orgnr_foretak'] = foretak['orgnr_foretak'].astype(str)
# industrivars['orgnr_foretak'] = industrivars['orgnr_foretak'].astype(str)

# # Remove leading/trailing spaces
# foretak['orgnr_foretak'] = foretak['orgnr_foretak'].str.strip()
# industrivars['orgnr_foretak'] = industrivars['orgnr_foretak'].str.strip()


# Check for common values
common_values = set(foretak['id']).intersection(set(industrivars['id']))
print(f"Number of common orgnr_foretak: {len(common_values)}")


In [ ]:
foretak_filtered = foretak[foretak['id'].isin(industrivars['id'])]
foretak_filtered.shape

In [ ]:
# perform left merge on 'id'
industrivars = pd.merge(industrivars,
    foretak_filtered, on=['id'], how='left'
)

industrivars.head()

In [ ]:
# Define the function to calculate 'andelen'
def calculate_andelen(row):
    if row['_har_indr'] == 0:
        return row['tmp_ny_driftsk'] / row['foretak_driftskostnad'] if row['foretak_driftskostnad'] != 0 else 0
    elif row['_har_indr'] == 1 and row['_industribedrift'] == 0:
        return 0
    elif row['_har_indr'] == 1 and row['_industribedrift'] == 1:
        return row['tmp_ny_driftsk'] / row['fdk_indr'] if row['fdk_indr'] != 0 else 0
    return 0

# Apply the function to each row in the DataFrame
industrivars['andelen'] = industrivars.apply(calculate_andelen, axis=1)

industrivars.head()

In [ ]:
# Step 1: Create the 'dk' column
industrivars['dk'] = industrivars['tmp_ny_driftsk']

# Step 2: Create the 'x' column by summing the specified columns
industrivars['x'] = industrivars[['gjeldende_lonn_kr', 'bedr_forbruk', 'no4295', 'no4995', 'no6200']].sum(axis=1)

# Step 3: Initialize the 'dknye' column with the value of 'dk'
industrivars['dknye'] = industrivars['dk']

# Step 4: Create the 'tyv' column and set it to 0 by default
industrivars['tyv'] = 0

# Step 5: Apply the condition to update 'dknye' and 'tyv'
condition = (industrivars['x'] > industrivars['dk'])
industrivars.loc[condition, 'dknye'] = industrivars['x']
industrivars.loc[condition, 'tyv'] = 1

# sort the DataFrame
industrivars = industrivars.sort_values(by=['id', 'lopenr', 'radnr']).reset_index(drop=True)


industrivars.head()

In [ ]:
filtered_df = industrivars[(industrivars['tyv'] == 1)]

tyvers = filtered_df[['id', 'lopenr']]
# drop duplicates for id
tyvers = tyvers.drop_duplicates(subset=['id'])

tyvers.head()

In [ ]:
industrivars = industrivars[industrivars['id'].isin(tyvers['id'])]

industrivars = industrivars.sort_values(by=['orgnr_foretak', 'lopenr', 'tyv']).reset_index(drop=True)

industrivars.head()

In [ ]:
# Calculate the 'utskudd' column
industrivars['utskudd'] = industrivars[['dk', 'gjeldende_lonn_kr', 'bedr_forbruk', 'no4295_f', 'no4995_f', 'no6200_f']].apply(lambda x: x[0] - x[1:].sum(), axis=1)
industrivars['utskudd'] = industrivars['utskudd'].abs()

# Replace missing values in 'tyv' with 0
industrivars['tyv'] = industrivars['tyv'].fillna(0)

# Keep only the specified columns
industrivars = industrivars[['orgnr_foretak', 'lopenr', 'radnr', 'utskudd', 'dk', 'dknye', 'tyv']]

# Display the resulting DataFrame
industrivars.head()

In [ ]:
sum7bb = industrivars.groupby(['orgnr_foretak', 'lopenr', 'tyv'], as_index=False)['utskudd'].sum()


# Pivot the table to create a wide format
sum7bb_pivot = sum7bb.pivot(index=['orgnr_foretak', 'lopenr'], columns='tyv', values='utskudd').reset_index()

# Rename the columns
sum7bb_pivot = sum7bb_pivot.rename(columns={0: 'tyv0', 1: 'tyv1'})

# Merge the pivoted sums back into the original industrivars DataFrame
merged_df = pd.merge(industrivars, sum7bb_pivot, on=['orgnr_foretak', 'lopenr'], how='inner')

# Initialize andel1, andel2 with NaN
merged_df['andel1'] = np.nan
merged_df['andel2'] = np.nan

# Update andel1 where tyv == 0
merged_df.loc[merged_df['tyv'] == 0, 'andel1'] = merged_df['utskudd'] / merged_df['tyv0']

# Update andel2 where tyv == 0
merged_df.loc[merged_df['tyv'] == 0, 'andel2'] = np.round(merged_df['andel1'] * merged_df['tyv1'])

# Update tmp_ny_driftsk based on conditions
merged_df.loc[merged_df['tyv'] == 0, 'tmp_ny_driftsk'] = merged_df['dknye'] - merged_df['andel2']
merged_df.loc[merged_df['tyv'] == 1, 'tmp_ny_driftsk'] = merged_df['dknye']

# 

# Keep only the specified columns
filtered_merged_df = merged_df[['orgnr_foretak', 'lopenr', 'radnr', 'tmp_ny_driftsk']]


filtered_merged_df.head(50)


In [ ]:
merged_df = pd.merge(enhetene_brukes7c, filtered_merged_df, on=['orgnr_foretak', 'lopenr', 'radnr'], how='left', indicator=True)

print(merged_df.shape)
merged_df.head()

In [ ]:
# Create the 'new_drkost' column based on the value of '_merge'

# fill tmp_ny_driftsk_y NaN with 0
merged_df['tmp_ny_driftsk_y'] = merged_df['tmp_ny_driftsk_y'].fillna(0)

merged_df['tmp_ny_driftsk'] = np.where(merged_df['_merge'] == 'left_only', 
                                   merged_df['tmp_ny_driftsk_x'], 
                                   np.where(merged_df['_merge'] == 'both', 
                                            merged_df['tmp_ny_driftsk_y'], 
                                            np.nan))


merged_df.loc[merged_df['regtype'] == '01', 'new_oms'] = merged_df['foretak_omsetning']
merged_df.loc[merged_df['regtype'] == '01', 'new_drkost'] = merged_df['foretak_driftskostnad']
merged_df.loc[merged_df['regtype'] == '01', 'tmp_ny_driftsk'] = merged_df['foretak_driftskostnad']
merged_df.loc[merged_df['regtype'] == '01', 'bedr_salgsint'] = merged_df['salgsint']
merged_df.loc[merged_df['regtype'] == '01', 'bedr_forbruk'] = merged_df['forbruk']

In [ ]:


grouped = (
    merged_df.groupby("id")[["tmp_ny_driftsk"]].sum().reset_index()
)

grouped.rename(
    columns={"tmp_ny_driftsk": "tot_drkost_fordelt"},
    inplace=True,
)

final_adjustment = pd.merge(merged_df, grouped, on="id", how="left")

final_adjustment['diff'] = final_adjustment['foretak_driftskostnad'] - final_adjustment['tot_drkost_fordelt']

# sort difference in descending order
final_adjustment = final_adjustment.sort_values(by="diff", ascending=True)

final_adjustment.head(50)

In [ ]:
# count NaN values for new_drkost
merged_df['tmp_ny_driftsk'].isna().sum()

# # create new df where new_drkost is NaN
test = merged_df[merged_df['tmp_ny_driftsk'].isna()]

# 'new_drkost' = merged_df['tmp_ny_driftsk']

merged_df['new_drkost'] = merged_df['tmp_ny_driftsk']

merged_df["tmp_sn2007_5"] = merged_df["naring"]

In [ ]:
# fill bedr_forbruk Nan with 0
merged_df['bedr_forbruk'] = merged_df['bedr_forbruk'].fillna(0)
merged_df['bedr_salgsint'] = merged_df['bedr_salgsint'].fillna(0)
merged_df['oms'] = merged_df['oms'].fillna(0)
merged_df['new_drkost'] = merged_df['new_drkost'].fillna(0)

merged_df["n3"] = merged_df["tmp_sn2007_5"].str[:4]
# merged_df["lonn"] = merged_df["lonn"].str.replace(',', '').astype(float)
merged_df["lonn_pst_aordn"] = merged_df["lonn_pst_aordn"].str.replace(',', '').astype(float)

# convert b_sysselsetting_syss  to int
merged_df["b_sysselsetting_syss"] = merged_df["b_sysselsetting_syss"].astype(int)
merged_df["fjor_driftskost_kr_t1"] = merged_df["fjor_driftskost_kr_t1"].astype(int)
merged_df["fjor_lonn_kr_t1"] = merged_df["fjor_lonn_kr_t1"].astype(int)
merged_df["fjor_omsetn_kr_t1"] = merged_df["fjor_omsetn_kr_t1"].astype(int)
merged_df["fjor_snittlonn_t1"] = merged_df["fjor_snittlonn_t1"].astype(int)
merged_df["fjor_snittoms_t1"] = merged_df["fjor_snittoms_t1"].astype(int)
merged_df["fjor_syssel_t1"] = merged_df["fjor_syssel_t1"].astype(int)
merged_df["tmp_forbruk_bed"] = merged_df["tmp_forbruk_bed"].astype(int)
merged_df["tmp_ny_bdr_syss"] = merged_df["tmp_ny_bdr_syss"].astype(int)
merged_df["tmp_salgsint_bed"] = merged_df["tmp_salgsint_bed"].astype(int)
merged_df["tmp_snittlonn"] = merged_df["tmp_snittlonn"].astype(int)
merged_df["tmp_snittoms"] = merged_df["tmp_snittoms"].astype(int)

merged_df["fjor_nace_b_t1"] = merged_df["fjor_nace_b_t1"].astype(str)
merged_df["regtype"] = merged_df["regtype"].astype(str)
merged_df["tmp_sn2007_5"] = merged_df["tmp_sn2007_5"].astype(str)




print(merged_df.shape)
# drop duplicates
# merged_df = merged_df.drop_duplicates()
# print(merged_df.shape)

In [ ]:
merged_df["n3"] = merged_df["tmp_sn2007_5"].str[:4]
merged_df["n2"] = merged_df["tmp_sn2007_5"].str[:2]

merged_df.head()

In [ ]:
merged_df.to_parquet(
    f"gs://ssb-prod-noeku-data-produkt/temp/knn_varehandel_cleaned.parquet",
    storage_options={"token": AuthClient.fetch_google_credentials()},
)

In [ ]:
timeseries_knn.to_parquet(
    f"gs://ssb-prod-noeku-data-produkt/temp/timeseries_knn.parquet",
    storage_options={"token": AuthClient.fetch_google_credentials()},
)

In [ ]:
# print option to show all columns
pd.set_option("display.max_columns", None)

# print out all columns list in time_series_df 
# print(time_series_df.columns)
# print(merged_df.columns)

time_series_df["n3"] = time_series_df["tmp_sn2007_5"].str[:4]
time_series_df["n2"] = time_series_df["tmp_sn2007_5"].str[:2]
merged_df["n2"] = merged_df["tmp_sn2007_5"].str[:2]

temp_1 = time_series_df[['id',
                         'nacef_5',
                         'orgnr_n_1',
                         'b_sysselsetting_syss',
                         'b_kommunenr',
                         'gjeldende_lonn_kr', 
                         'gjeldende_driftsk_kr',
                         'gjeldende_omsetn_kr',
                         'tmp_forbruk_bed',
                         'tmp_salgsint_bed',
                         'tmp_sn2007_5',
                         'n3',
                         'n2',
                         'year']]

# rename columns
temp_1 = temp_1.rename(columns={'b_sysselsetting_syss':'syss',
                                'b_kommunenr':'kommunenr',
                                'gjeldende_lonn_kr':'lonn',
                                'gjeldende_omsetn_kr':'oms',
                                'gjeldende_driftsk_kr': 'drkost',
                                'tmp_forbruk_bed':'forbruk',
                                'tmp_salgsint_bed':'salgsint',
                               })

temp_1 = temp_1[temp_1['year'] != 2021]

temp_2 = merged_df[['id',
                 'nacef_5',
                 'orgnr_n_1',
                 'b_sysselsetting_syss',
                 'b_kommunenr',
                 'gjeldende_lonn_kr', 
                 'new_drkost',
                 'oms',
                 'bedr_forbruk',
                 'bedr_salgsint',
                 'tmp_sn2007_5',
                 'n3',
                 'n2',
                 'year']]

temp_2 = temp_2.rename(columns={'b_sysselsetting_syss':'syss',
                                'b_kommunenr':'kommunenr',
                                'gjeldende_lonn_kr':'lonn',
                                'bedr_forbruk':'forbruk',
                                'bedr_salgsint':'salgsint',
                                'new_drkost': 'drkost'
                               })

temp_2 = temp_2[temp_2['year'] == 2021]

In [ ]:
# fill NaN for forbruk and salgint for temp_1
temp_1['forbruk'] = temp_1['forbruk'].fillna(0)
temp_1['salgsint'] = temp_1['salgsint'].fillna(0)
temp_2['forbruk'] = temp_2['forbruk'].fillna(0)
temp_2['salgsint'] = temp_2['salgsint'].fillna(0)

In [ ]:
temp_2.head()

In [ ]:
timeseries_knn = pd.concat([temp_1, temp_2], axis=0)

# timeseries_knn['resultat'] = timeseries_knn['oms'] - timeseries_knn['drkost']

In [ ]:
timeseries_knn.dtypes

In [ ]:
# aggregate forbruk per year

columns_to_convert = ['salgsint', 'forbruk', 'oms', 'drkost', 'lonn', 'syss']

# Convert columns to integers using pd.to_numeric for safe conversion, errors='coerce' will set issues to NaN
for column in columns_to_convert:
    timeseries_knn[column] = pd.to_numeric(timeseries_knn[column], errors='coerce')

timeseries_knn['year'] = timeseries_knn['year'].astype(str)
timeseries_knn['n3'] = timeseries_knn['n3'].astype(str)

timeseries_knn['resultat'] = timeseries_knn['oms'] - timeseries_knn['drkost']



# filter for n3 in 45, 46 or 47
timeseries_knn = timeseries_knn[timeseries_knn['n2'].isin(['45', '46', '47'])]
temp = timeseries_knn.copy()
timeseries_knn_agg = timeseries_knn.groupby(["year", "n3"])[["forbruk", "oms", "drkost", "salgsint", "lonn", 'syss', "resultat"]].sum().reset_index()
# timeseries_knn_agg = timeseries_knn.groupby(["year", "n3"])[["forbruk", "oms", "drkost", "salgsint", "lonn", 'syss', "resultat"]].sum().reset_index()
timeseries_knn_agg['lonn_pr_syss'] = timeseries_knn_agg['lonn'] / timeseries_knn_agg['syss']
timeseries_knn_agg['oms_pr_syss'] = timeseries_knn_agg['oms'] / timeseries_knn_agg['syss']
timeseries_knn_agg["n2"] = timeseries_knn_agg["n3"].str[:2]



timeseries_knn__kommune_agg = temp.groupby(["year", "kommunenr", "n3"])[["forbruk", "oms", "drkost", "salgsint", "lonn", 'syss', "resultat"]].sum().reset_index()
timeseries_knn__kommune_agg['lonn_pr_syss'] = timeseries_knn__kommune_agg['lonn'] / timeseries_knn_agg['syss']
timeseries_knn__kommune_agg['oms_pr_syss'] = timeseries_knn__kommune_agg['oms'] / timeseries_knn_agg['syss']
timeseries_knn__kommune_agg["n2"] = timeseries_knn__kommune_agg["n3"].str[:2]

In [ ]:
# timeseries_knn__kommune_agg.head(50)

# filter for when year = 2021

test = timeseries_knn_agg[timeseries_knn_agg['year'] == '2021']
timeseries_knn__kommune_agg.head(15)

In [ ]:
# fil_path = [
#     f
#     for f in fs.glob(
#         f"gs://ssb-prod-noeku-data-produkt/temp/timeseries_knn.parquet"
#     )
#     if f.endswith(".parquet")
# ]

# # Use the ParquetDataset to read multiple files
# dataset = pq.ParquetDataset(fil_path, filesystem=fs)
# table = dataset.read()

# # Convert to Pandas DataFrame
# timeseries_knn_agg = table.to_pandas()



In [ ]:
timeseries_knn__kommune_agg.to_parquet(
    f"gs://ssb-prod-noeku-data-produkt/temp/timeseries_knn_kommune.parquet",
    storage_options={"token": AuthClient.fetch_google_credentials()},
)

In [ ]:
timeseries_knn_agg.head()

In [ ]:
grouped_df = merged_df.groupby('id').agg({
    'oms': 'sum',
    'new_drkost': 'sum'
}).reset_index()


# Merging the sums back with the original to compare
comparison_df = pd.merge(merged_df, grouped_df, on='id', suffixes=('', '_sum'))


# Check if the summed 'new_oms' and 'new_drkost' match 'foretak_omsetning' and 'foretak_driftskostnad'
comparison_df['oms_match'] = comparison_df['foretak_omsetning'] == comparison_df['oms_sum']
comparison_df['drkost_match'] = comparison_df['foretak_driftskostnad'] == comparison_df['new_drkost_sum']


# Print rows where totals do not match
# print(comparison_df[~(comparison_df['oms_match'] & comparison_df['drkost_match'])])

check = comparison_df[~(comparison_df['oms_match'] & comparison_df['drkost_match'])]

# Optionally, print out all rows to manually inspect
# print(comparison_df)
check['oms_diff'] = check['foretak_omsetning'] - check['oms_sum']
check['drkost_diff'] = check['foretak_driftskostnad'] - check['new_drkost_sum']

# filter for when either oms_diff or drkost_diff is not 0
check[(check['oms_diff'] != 0) | (check['drkost_diff'] != 0)]

# sort oms_diff
check.sort_values(by="oms_diff", ascending=True, inplace=True)

check.head()

In [ ]:
#### TEST RETTES

#### TEST
test_grouped = (
    merged_df.groupby("id")[["new_drkost"]].sum().reset_index()
)

test_grouped.rename(
    columns={"new_drkost": "test_tot_drkost_fordelt"},
    inplace=True,
)

test1 = pd.merge(merged_df, test_grouped, on="id", how="left")

test1['drkost_diff'] = test1['foretak_driftskostnad'] - test1['test_tot_drkost_fordelt']

test1 = test1.sort_values(by='drkost_diff', ascending=True)

test_grouped = (
    merged_df.groupby("id")[["new_oms"]].sum().reset_index()
)

test_grouped.rename(
    columns={"new_oms": "test_tot_oms_fordelt"},
    inplace=True,
)

test2 = pd.merge(merged_df, test_grouped, on="id", how="left")

test2['oms_diff'] = test2['foretak_omsetning'] - test2['test_tot_oms_fordelt']

test2 = test2.sort_values(by='oms_diff', ascending=True)

# change pd option to show all rows 
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

In [ ]:
test2.head(50)

In [ ]:
merged_df.head()